### Chaotic dynamical systems
This notebook illustrates two ordinary differential equations that exhibit chaotic behavior.

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.integrate import solve_ivp

def lorenz(t, u, *pars):
    sigma = pars[0]
    rho = pars[1]
    beta = pars[2]
    f = [ sigma*(u[1]-u[0]), rho*u[0]-u[1]-u[0]*u[2], u[0]*u[1]-beta*u[2] ]
    return f

def roessler(t, u, *pars):
    a = pars[0]
    b = pars[1]
    c = pars[2]
    f = [ -u[1]-u[2], u[0]+a*u[1], b-c*u[2]+u[0]*u[2] ]
    return f

#### Lorenz system
The Lorenz system is given by
$$
\begin{align}
\dot{x} & = \sigma (y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = bxy - \beta z
\end{align}
$$
where $(\sigma,\rho,\beta)=(10, 28, 8/3)$ will result in chaotic dynamics.

In [2]:
sigma = 10
rho = 28
beta = 8/3

u0 = [2, 8, 23.5]
T = 100
N = 10

pars = (sigma, rho, beta)
t = np.linspace(0, T, int(100*T))

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(1,0,N)))
for n in range(N):
    u0 = [2, 8, 23.5] + np.random.uniform(-0.5, 0.5, 3)
    soln = solve_ivp(lorenz, [0, T], u0, t_eval=t, args=pars)
    ax.plot(soln.y[0], soln.y[1], soln.y[2], linewidth=0.5, alpha=0.4)
ax.set_title('Lorenz system')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Solutions to the Lorenz system in the chaotic regime depend very sensitively on the initial data. The simulation below shows graphs of the x-components of two solutions for the same parameter values whose initial conditions differ by 0.0001.

In [3]:
T = 40
u1 = np.array([2, 8, 23.5])
u2 = u1 + np.array([0.0001, 0, 0])

pars = (10, 28, 8/3)
t = np.linspace(0, T, int(100*T))

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
soln = solve_ivp(lorenz, [0, T], u1, t_eval=t, args=pars)
ax.plot(soln.t, soln.y[0], linewidth=0.5, color="tab:blue")
soln = solve_ivp(lorenz, [0, T], u2, t_eval=t, args=pars)
ax.plot(soln.t, soln.y[0], linewidth=0.5, color="tab:red")
ax.set_title('Lorenz system: Sensitive dependence on initial data')
ax.set_xlabel('t')
ax.set_ylabel('x')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Rössler system
The Rössler system is given by
$$
\begin{align}
\dot{x} & = -y-z \\
\dot{y} & = x+ay \\
\dot{z} & = b-cz+xz
\end{align}
$$
with $a,b=0.1$, where $c$ varies between 4 and 10.

In [72]:
a = 0.1
b = 0.1
c = 9

u0 = [1, 1, 1]
T = 300
N = 10

pars = (a, b, c)
t = np.linspace(0, T, int(100*T))

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.plasma(np.linspace(0,1,N)))
soln = solve_ivp(roessler, [0, 5000], u0, args=pars)
u0 = soln.y[0:3,-1]
for n in range(N):
    u1 = u0 + np.random.uniform(-0.01, 0.01, 3)
    soln = solve_ivp(roessler, [0, T], u1, t_eval=t, args=pars)
    ax.plot(soln.y[0], soln.y[1], soln.y[2], linewidth=0.5, alpha=0.25)
ax.set_title('Rössler system')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As the parameter c increases from 4 to 9, the Rössler system exhibits a period-doubling cascade that ultimates generates chaotic attaractors for sufficiently large values of c. We visualize these bifurcations by plotting x(t) for values of t where y(t) crosses through 0 as a function of the parameter c. For c close to 4, there is just one such value attained by the solution, so the solution winds around once; as c increases, subsequent period-doubling bifurcations are encountered near c=5.4 and c=7.8. 

In [73]:
u0 = [1,1,1]
c = np.arange(4, 10, 0.05)
pars = [0.1, 0.1, c[0]]
T = 200

def cross_section(t, y, *pars): return y[1]
cross_section.direction = 1

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
soln = solve_ivp(roessler, [0, 4000], u0, args=pars)
for cvalue in c:
    pars[2] = cvalue
    soln = solve_ivp(roessler, [0, 500], soln.y[0:3,-1], args=pars)
    soln = solve_ivp(roessler, [0, T], soln.y[0:3,-1], args=pars, events=cross_section, rtol=1.0e-8)
    ax.scatter(cvalue+0*soln.y_events[0][:,0], soln.y_events[0][:,0], s=2, color="tab:blue")
ax.set_title('Rössler system: Bifurcation diagram')
ax.set_xlabel('c')
ax.set_ylabel('x @y=0')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
u0 = [1,1,1]
c = [4.5, 7, 8.2]
pars = [0.1, 0.1, c[0]]
T = 35

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab10(np.linspace(0,1,10)))
soln = solve_ivp(roessler, [0, 1], u0, args=pars)
for cvalue in c:
    pars[2] = cvalue
    soln = solve_ivp(roessler, [0, 4000], soln.y[0:3,-1], args=pars)
    soln = solve_ivp(roessler, [0, T], soln.y[0:3,-1], args=pars, rtol=1.0e-8)
    ax.plot(soln.t, soln.y[0], linewidth=2)
ax.set_title('Rössler system: Periodic solutions')
ax.set_xlabel('t')
ax.set_ylabel('x')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …